<a href="https://colab.research.google.com/github/huangwenbing4github/keras_demo/blob/main/pytorch-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [33]:
train = datasets.FashionMNIST(root='data', train=True, download=True, transform=ToTensor())
test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=ToTensor())

In [20]:
batch_size = 64
train_dataloader = DataLoader(train, batch_size=batch_size)
test_dataloader = DataLoader(test, batch_size=batch_size)

for X,y in test_dataloader:
  print(f'Shape of X[N,C,H,W]: {X.shape}')
  print(f'Shape of y:{y.shape},{y.dtype}')
  break

Shape of X[N,C,H,W]: torch.Size([64, 1, 28, 28])
Shape of y:torch.Size([64]),torch.int64


In [21]:
device = ('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(device)


cpu


In [22]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [23]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [24]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X,y) in enumerate(dataloader):
    X,y = X.to(device), y.to(device)
    pred = model(X)
    loss = loss_fn(pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if batch % 100 == 0:
      loss,current = loss.item,(batch+1)*len(X)
      print(f'loss:{loss}, {current}, {size}')

In [25]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0,0
  with torch.no_grad():
    for X, y in dataloader:
      X,y = X.to(device),y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f'test error: accuracy {100*correct}, avg_loss: {test_loss}')

In [26]:
epochs = 5
for t in range(epochs):
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print('done')

loss:<built-in method item of Tensor object at 0x7ffb0602a0e0>, 64, 60000
loss:<built-in method item of Tensor object at 0x7ffb05e4edb0>, 6464, 60000
loss:<built-in method item of Tensor object at 0x7ffb05e4ee00>, 12864, 60000
loss:<built-in method item of Tensor object at 0x7ffb05e53130>, 19264, 60000
loss:<built-in method item of Tensor object at 0x7ffb05e82450>, 25664, 60000
loss:<built-in method item of Tensor object at 0x7ffb0602a4a0>, 32064, 60000
loss:<built-in method item of Tensor object at 0x7ffb05e5b360>, 38464, 60000
loss:<built-in method item of Tensor object at 0x7ffb05e82590>, 44864, 60000
loss:<built-in method item of Tensor object at 0x7ffb05e82770>, 51264, 60000
loss:<built-in method item of Tensor object at 0x7ffb0600bc70>, 57664, 60000
test error: accuracy 37.95, avg_loss: 2.1428447878284818
loss:<built-in method item of Tensor object at 0x7ffb05e5b4f0>, 64, 60000
loss:<built-in method item of Tensor object at 0x7ffb05e824f0>, 6464, 60000
loss:<built-in method item 

In [27]:
torch.save(model.state_dict(), 'model.pth')


In [30]:
reload = NeuralNetwork()
reload.load_state_dict(torch.load('model.pth'))


<All keys matched successfully>

In [35]:
clazz = [
    'T-shirt/top','Trouser','Pullover',
    'Dress','Coat','Sandal',
    'Shirt','Sneaker','Bag','Ankle boot'
]
reload.eval()
x,y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  pred = reload(x)
  predicted, actual = clazz[pred[0].argmax(0)],clazz[y]
  print(f'predict:{predicted},actual:{actual}')

predict:Ankle boot,actual:Ankle boot
